In [1]:
import sys
sys.path.insert(0, "../..")

from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.show_plot import show_plot
import numpy as np
import matplotlib

In [2]:
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, opt_set, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=opt_set
    )
    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 16})
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()):
            wall_times = results[coord]['wall_time'][opts]
            print("elapsed real time: ", wall_times)
            mean_time = np.nanmean(wall_times)
            print("mean time:", mean_time)
            plot_data = results[coord]['numerical'][opts]
            m_diff, n_diff = [], []
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                    
                    deltami, deltani = plotter.mass_difference(mnorm, plot_data[i], results[coord]['grid']['rh'])
                    m_diff.append(deltami)
                    n_diff.append(deltani)
                    
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'fct': True","+ fct"],
                            ["'iga': True","+ iga"],
                            ["'tot': True","+ tot"],
                            ["{",""],["}",""],[","," "]]                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    linewidth = linewidth,
                    mnorm=mnorm, color = colors[opt_i]
                )
        print(f"grid_layout: {coord}")
        show_plot(filename)

In [3]:
plot_setup = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(base=2), "GC_max":.26,"n_bins":75}
plot_setup2 = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07,"n_bins":75}

In [4]:
compute_and_plot(**plot_setup, opt_set=({"n_iters":1},), plots=('n','m'), filename = 'fig_upwinda.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   12.2s finished


elapsed real time:  [nan, 0.026143790849673203, 0.025806451612903226, 0.026611093299134338]
mean time: 0.026187111920570254
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
grid_layout: x_log_of_pn


In [5]:
compute_and_plot(**plot_setup2, opt_set=({"n_iters":1},), plots=['n','m'],  filename = 'fig_upwindb.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   10.7s finished


elapsed real time:  [nan, 0.02727272727272727, 0.027925531914893616, 0.027662105889351578]
mean time: 0.027620121692324157
Mass difference: 0.36793541750237413 dimensionless %
Number difference: 2.8095569687690517e-14 dimensionless %
Mass difference: 0.4738031524986308 dimensionless %
Number difference: 0.006836362361410145 dimensionless %
Mass difference: 0.5923321831645456 dimensionless %
Number difference: 0.027240110707723638 dimensionless %
Mass difference: 0.10668574082613347 dimensionless %
Number difference: 0.26764450429797615 dimensionless %
grid_layout: x_id


In [6]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2}), plots=['n'], filename = 'fig_mpdata.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   26.3s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   26.3s finished


elapsed real time:  [nan, 0.06427015250544663, 0.06379928315412187, 0.07213850593138826]
mean time: 0.06673598053031891
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
elapsed real time:  [nan, 0.06427015250544663, 0.06379928315412187, 0.07213850593138826]
mean time: 0.06673598053031891
grid_layout: x_log_of_pn


In [7]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':3}), plots=['n'], filename = 'fig_mpdatas.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   48.8s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   48.8s finished


elapsed real time:  [nan, 0.10239651416122005, 0.1032258064516129, 0.1038794485411991]
mean time: 0.10316725638467734
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
elapsed real time:  [nan, 0.10239651416122005, 0.1032258064516129, 0.1038794485411991]
mean time: 0.10316725638467734
elapsed real time:  [nan, 0.10239651416122005, 0.1032258064516129, 0.1038794485411991]
mean time: 0.10316725638467734
grid_layout: x_log_of_pn


In [8]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True}), plots=['n'], filename = 'fig_iga.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   39.4s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   39.4s finished


elapsed real time:  [nan, 0.06318082788671024, 0.06451612903225806, 0.0647643475472908]
mean time: 0.0641537681554197
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
elapsed real time:  [nan, 0.06318082788671024, 0.06451612903225806, 0.0647643475472908]
mean time: 0.0641537681554197
elapsed real time:  [nan, 0.06318082788671024, 0.06451612903225806, 0.0647643475472908]
mean time: 0.0641537681554197
grid_layout: x_log_of_pn


In [9]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   50.4s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   50.4s finished


elapsed real time:  [nan, 0.14596949891067537, 0.14695340501792115, 0.15870471304905417]
mean time: 0.15054253899255024
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
elapsed real time:  [nan, 0.14596949891067537, 0.14695340501792115, 0.15870471304905417]
mean time: 0.15054253899255024
elapsed real time:  [nan, 0.14596949891067537, 0.14695340501792115, 0.15870471304905417]
mean time: 0.15054253899255024
grid_layout: x_log_of_pn


In [10]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'third_order_terms':True}), plots=['n'], filename = 'fig_tot.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   41.5s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   41.5s finished


elapsed real time:  [nan, 0.0664488017429194, 0.06810035842293907, 0.06797050336646361]
mean time: 0.06750655451077403
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
elapsed real time:  [nan, 0.0664488017429194, 0.06810035842293907, 0.06797050336646361]
mean time: 0.06750655451077403
elapsed real time:  [nan, 0.0664488017429194, 0.06810035842293907, 0.06797050336646361]
mean time: 0.06750655451077403
grid_layout: x_log_of_pn


In [11]:
compute_and_plot(**plot_setup, opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
), plots=['n'], filename='fig_multiopt.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   51.4s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   51.4s finished


elapsed real time:  [nan, 0.2374727668845316, 0.24301075268817204, 0.24110291760179545]
mean time: 0.2405288123914997
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
elapsed real time:  [nan, 0.2374727668845316, 0.24301075268817204, 0.24110291760179545]
mean time: 0.2405288123914997
elapsed real time:  [nan, 0.2374727668845316, 0.24301075268817204, 0.24110291760179545]
mean time: 0.2405288123914997
grid_layout: x_log_of_pn
